In [1]:
import pandas as pd
import requests
import time
import random
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime
import concurrent.futures
import yfinance as yf

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
INPUT_FILE = "260128_Earnings.csv"               # 입력 파일명
OUTPUT_SUCCESS = "260128_해외빨간줄.xlsx"  # 통합 파일 (매출 + 영업이익)
OUTPUT_FAILED = "failed_tickers.xlsx"   # 실패 티커 저장 (수동 확인용)

FRESHNESS_THRESHOLD = 110  # 110일(약 3.5개월) 지났으면 '미반영' 처리
MAX_WORKERS = 4            # 병렬 처리 개수 (너무 높이면 차단됨)
NUM_QUARTERS = 17          # 가져올 분기 수

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}

# 숫자 변환 헬퍼 함수
def parse_money_string(value_str):
    if not isinstance(value_str, str): return value_str
    s = value_str.strip().replace(',', '')
    if s == '-': return 0
    try:
        if s.endswith('B'): return float(s[:-1]) * 1_000_000_000
        elif s.endswith('M'): return float(s[:-1]) * 1_000_000
        elif s.endswith('K'): return float(s[:-1]) * 1_000
        elif s.endswith('%'): return float(s[:-1])
        else: return float(s)
    except: return 0

# 분기 라벨 생성 (최신 분기 = 4Q25 기준)
def generate_quarter_labels(num_quarters):
    """
    최신 분기를 4Q25로 시작해서 역순으로 라벨 생성
    예: ['4Q25', '3Q25', '2Q25', '1Q25', '4Q24', '3Q24', ...]
    """
    labels = []
    year = 25
    quarter = 4
    
    for _ in range(num_quarters):
        labels.append(f"{quarter}Q{year}")
        quarter -= 1
        if quarter == 0:
            quarter = 4
            year -= 1
    
    return labels

# yfinance에서 산업 정보 가져오기
def get_industry(ticker):
    """yfinance를 사용해 티커의 산업(industry) 정보 가져오기"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        # 'industry' 또는 'sector' 정보 반환
        industry = info.get('industry', info.get('sector', 'N/A'))
        return industry
    except:
        return 'N/A'

# yfinance로 거래소 확인 및 Stock Analysis 거래소 코드 반환
def get_stock_analysis_exchange(ticker, company_name=None):
    """
    yfinance로 거래소 확인 후 Stock Analysis URL 형식 반환
    Returns: (sa_exchange_code, original_exchange, matched_ticker)
    """
    # yfinance 거래소 → Stock Analysis 거래소 코드 매핑
    exchange_map = {
        # 미국 거래소 (기본 URL)
        'NMS': None, 'NYQ': None, 'ASE': None, 'PCX': None, 'NGM': None, 'NAS': None,
        
        # 유럽
        'STO': 'sto', 'PAR': 'epa', 'AMS': 'ams', 'BRU': 'bru', 'LSE': 'lse', 'FRA': 'fra', 'SWX': 'swx',
        
        # 아시아
        'JPX': 'tyo', 'TYO': 'tyo', 'HKG': 'hkg', 'KSC': 'ksc', 'KOE': 'koe', 
        'TAI': 'tai', 'SHH': 'shh', 'SHZ': 'shz', 'IDX': 'idx', 'NSI': 'nse', 'BOM': 'bse',
        
        # 기타
        'BMV': 'bmv', 'SAO': 'sao', 'JNB': 'jse', 'TAD': 'tase',
    }
    
    # 스마트 우선순위: 티커 패턴 기반
    search_attempts = [ticker]  # 원본
    
    # 숫자만 있는 티커 (아시아 우선)
    if ticker.replace(' ', '').replace('-', '').isdigit():
        ticker_clean = ticker.replace(' ', '')
        if len(ticker_clean) <= 4:
            # 4자리 이하 → 도쿄 우선
            search_attempts.extend([f"{ticker_clean}.T", f"{ticker_clean}.HK"])
        else:
            # 5자리 이상 → 홍콩, 인도, 중동
            search_attempts.extend([f"{ticker_clean}.HK", f"{ticker_clean}.NS", f"{ticker_clean}.BO", f"{ticker_clean}.SR"])
    
    # 공백 포함 티커 (스웨덴 클래스주)
    elif ' ' in ticker:
        ticker_hyphen = ticker.replace(' ', '-')
        search_attempts.extend([f"{ticker_hyphen}.ST", f"{ticker}.ST"])
    
    # 한국 주식은 검색하지 않음
    # 대문자 단어 티커 (인도 우선 - 티커명이 곧 기업명)
    elif ticker.isupper() and len(ticker) > 4:
        search_attempts.extend([f"{ticker}.NS", f"{ticker}.BO"])
    
    # 일반 티커 - 주요 거래소만 (한국 제외)
    else:
        search_attempts.extend([
            f"{ticker}.ST",  # Stockholm
            f"{ticker}.PA",  # Paris
            f"{ticker}.L",   # London
            f"{ticker}.T",   # Tokyo
            f"{ticker}.HK",  # Hong Kong
        ])
    
    # 각 티커 시도
    for attempt_ticker in search_attempts:
        try:
            stock = yf.Ticker(attempt_ticker)
            info = stock.info
            
            # 유효한 데이터 확인
            symbol = info.get('symbol')
            exchange = info.get('exchange', '')
            
            # 데이터가 있고 거래소 정보가 있으면 성공
            if symbol and exchange:
                sa_code = exchange_map.get(exchange)
                return sa_code, exchange, attempt_ticker
                
        except Exception as e:
            continue
    
    # 모든 시도 실패
    return None, None, ticker

# ==========================================
# 2. 개별 기업 처리 함수 (Worker)
# ==========================================
def process_ticker(ticker_data):
    # ticker_data는 (ticker, company_name) 튜플
    if isinstance(ticker_data, tuple):
        raw_ticker, company_name = ticker_data
    else:
        raw_ticker = ticker_data
        company_name = None
    
    # 티커 전처리
    ticker = str(raw_ticker).strip().replace('.', '-').replace(' ', '-').lower()
    
    # yfinance로 거래소 확인 (기업명 포함)
    sa_exchange, original_exchange, matched_ticker = get_stock_analysis_exchange(raw_ticker, company_name)
    
    # 한국 거래소는 제외 (별도 툴 사용)
    if sa_exchange in ['ksc', 'koe']:
        return 'SKIP_KOREA', original_exchange, matched_ticker
    
    # 거래소 정보 로깅
    exchange_info = f"{original_exchange} → {sa_exchange if sa_exchange else 'US'}"
    if matched_ticker != raw_ticker:
        exchange_info += f" (matched: {matched_ticker})"
    
    # 한국 주식 필터링 확인
    if exchange_info == 'SKIP_KOREA':
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Korean Stock (Skipped - separate tool)'}
    
    # 숫자만 있는 티커 중 한국 가능성 체크 (A로 시작하는 6자리)
    if isinstance(raw_ticker, str) and raw_ticker.startswith('A') and len(raw_ticker) == 7 and raw_ticker[1:].isdigit():
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Korean Stock Code (Skipped)'}
    
    # 한국 주식/숫자만 있는 티커 필터링 (단, 일본 주식은 허용)
    if sa_exchange not in ['tyo', None]:  # 일본이나 미국이 아니면
        if any(char.isdigit() for char in ticker) and not ticker.isalpha():
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Non-supported number ticker ({exchange_info})'}

    # Stock Analysis URL 생성 (거래소별)
    if sa_exchange:
        # 해외 거래소 - matched_ticker 사용 (접미사 포함 가능)
        # Stock Analysis는 접미사 없이 사용하므로 원본 티커 사용
        url = f"https://stockanalysis.com/quote/{sa_exchange}/{raw_ticker.upper().replace(' ', '-')}/financials/?p=quarterly"
    else:
        # 미국 거래소 (기본)
        url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"
    
    retry_count = 0
    time.sleep(random.uniform(1.0, 3.0)) # 기본 대기

    # 접속 시도
    while retry_count < 3:
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code == 200:
                break
            elif response.status_code == 404:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'404 Not Found ({exchange_info})'}
            elif response.status_code == 429: # 과부하 시 대기
                time.sleep(random.uniform(10, 20))
                retry_count += 1
            else:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Error {response.status_code}'}
        except:
            retry_count += 1
            time.sleep(2)
            
    if retry_count >= 3:
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Connection Timeout'}

    # 데이터 파싱
    try:
        dfs = pd.read_html(StringIO(response.text))
        if not dfs: return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'No Table Found'}
        df_fin = dfs[0]

        # 날짜 컬럼 확인
        date_cols = df_fin.columns[1:].tolist()
        latest_date_str = date_cols[0]
        
        # 날짜 및 연간 데이터 검증
        freshness_diff = 0
        is_before_4q25 = False
        try:
            d1 = pd.to_datetime(latest_date_str, format='mixed')
            d2 = pd.to_datetime(date_cols[1], format='mixed')
            
            # 1. 연간 데이터(FY) 체크: 간격이 250일 이상이면 실패 처리
            if abs((d1 - d2).days) > 250:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Received Annual Data (FY)'}
            
            # 2. 최신성 체크
            freshness_diff = (datetime.now() - d1).days
            
            # 3. 4Q25 이전 체크 - Latest_Date가 "Q4 2025" 형식인지 확인
            date_str_lower = latest_date_str.lower()
            if 'q4' in date_str_lower and '2025' in date_str_lower:
                is_before_4q25 = False
            elif '2025' in date_str_lower:
                is_before_4q25 = True  # Q1, Q2, Q3 2025
            elif '2024' in date_str_lower or int(latest_date_str.split('-')[0]) < 2025:
                is_before_4q25 = True
            else:
                is_before_4q25 = False
        except:
            pass # 날짜 파싱 에러나도 일단 진행

        # ========================================
        # 매출 데이터 찾기 (정확히 일치하는 경우만)
        # ========================================
        revenue_target = ["Revenue", "Total Revenue", "Net Revenue", "Sales"]
        revenue_row = pd.DataFrame()
        for metric in revenue_target:
            # 정확히 일치하는 경우만 찾기 (strip으로 공백 제거 후 비교)
            temp = df_fin[df_fin.iloc[:, 0].str.strip().str.lower() == metric.lower()]
            if not temp.empty: 
                revenue_row = temp
                break
        
        if revenue_row.empty:
            revenue_values = None
            revenue_growth = 0
            revenue_avg = 0
        else:
            revenue_values = [parse_money_string(v) for v in revenue_row.iloc[0, 1:].tolist()]
            if len(revenue_values) >= 5:
                recent_avg = sum(revenue_values[0:4]) / 4
                past_avg = sum(revenue_values[1:5]) / 4
                revenue_growth = (recent_avg / past_avg) - 1 if past_avg != 0 else 0
                revenue_avg = recent_avg
            else:
                revenue_growth = 0
                revenue_avg = 0

        # ========================================
        # 영업이익 데이터 찾기
        # ========================================
        op_target = ["Operating Income", "Operating Profit", "Pretax Income", "Net Income"]
        op_row = pd.DataFrame()
        for metric in op_target:
            temp = df_fin[df_fin.iloc[:, 0].str.contains(metric, case=False, na=False)]
            if not temp.empty: 
                op_row = temp
                break
        
        if op_row.empty:
            op_values = None
            op_growth = 0
            op_avg = 0
        else:
            op_values = [parse_money_string(v) for v in op_row.iloc[0, 1:].tolist()]
            if len(op_values) >= 5:
                recent_avg = sum(op_values[0:4]) / 4
                past_avg = sum(op_values[1:5]) / 4
                op_growth = (recent_avg / past_avg) - 1 if past_avg != 0 else 0
                op_avg = recent_avg
            else:
                op_growth = 0
                op_avg = 0

        # 둘 다 없으면 실패
        if revenue_values is None and op_values is None:
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'No Revenue or OpIncome Data'}
        
        # 분기 수 체크
        if revenue_values and len(revenue_values) < NUM_QUARTERS:
            revenue_values = None
        if op_values and len(op_values) < NUM_QUARTERS:
            op_values = None
            
        # 둘 다 분기 부족이면 실패
        if revenue_values is None and op_values is None:
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Data Shortage (<{NUM_QUARTERS}Q)'}

        # 성공 데이터 반환
        result = {
            'status': 'success',
            'ticker': raw_ticker.upper(),
            'industry': 'PENDING',
            'latest_date': latest_date_str,
            'is_before_4q25': is_before_4q25,
            'revenue_values': revenue_values[:NUM_QUARTERS] if revenue_values else None,
            'revenue_growth': revenue_growth,
            'revenue_avg': revenue_avg,
            'op_values': op_values[:NUM_QUARTERS] if op_values else None,
            'op_growth': op_growth,
            'op_avg': op_avg,
        }
        
        print(f"[{raw_ticker.upper()}] 성공 ({exchange_info}) - 매출: {revenue_growth*100:.1f}%, 영업이익: {op_growth*100:.1f}%)")
        return result

    except Exception as e:
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': str(e)}

# ==========================================
# 3. 데이터프레임 생성 함수
# ==========================================
def create_dataframe(success_data, data_type='revenue'):
    """
    data_type: 'revenue' 또는 'operating'
    """
    rows = []
    quarter_labels = generate_quarter_labels(NUM_QUARTERS)
    quarter_labels_reversed = list(reversed(quarter_labels))
    
    for data in success_data:
        if data_type == 'revenue':
            if data['revenue_values'] is None:
                continue
            values = data['revenue_values']
            growth = data['revenue_growth']
            avg = data['revenue_avg']
        else:  # operating
            if data['op_values'] is None:
                continue
            values = data['op_values']
            growth = data['op_growth']
            avg = data['op_avg']
        
        # 순서 반전: 최근 분기를 오른쪽에
        values_reversed = list(reversed(values))
        
        row = {
            'Ticker': data['ticker'],
            'Industry': data['industry'],
            'Latest_Date': data['latest_date'],
            'Growth_Rate': growth * 100,
            'Recent_Avg': avg,
            'Is_Before_4Q25': data['is_before_4q25'],
        }
        
        # 분기 데이터 추가
        for i, q_label in enumerate(quarter_labels_reversed):
            if i < len(values_reversed):
                row[q_label] = values_reversed[i]
        
        rows.append(row)
    
    return pd.DataFrame(rows)

# ==========================================
# 4. 스타일 적용 함수
# ==========================================
def apply_styling(df_input):
    """롤링 성장률 체크 및 스타일 적용"""
    df = df_input.copy()
    
    # 롤링 성장률 체크 함수
    def check_rolling_growth(row):
        highlight_quarters = set()
        
        try:
            q_cols = [col for col in row.index if 'Q' in col and col[0].isdigit()]
            n = len(q_cols)
            
            if n >= 5:
                for position in range(0, n - 4):
                    current_idx = n - 1 - position
                    recent_4q = [row[q_cols[current_idx - i]] for i in range(4)]
                    prev_4q = [row[q_cols[current_idx - 1 - i]] for i in range(4)]
                    
                    recent_avg = sum(recent_4q) / 4
                    prev_avg = sum(prev_4q) / 4
                    
                    if prev_avg != 0:
                        growth_ratio = recent_avg / prev_avg
                        if growth_ratio >= 1.1:
                            highlight_quarters.add(q_cols[current_idx])
        except:
            pass
        
        return highlight_quarters
    
    # 각 행에 하이라이트할 분기 저장
    highlight_data = df.apply(check_rolling_growth, axis=1)
    
    # 스타일 함수
    def style_growth_cells(row):
        styles = ['' for _ in row.index]
        row_idx = row.name
        
        if row_idx in highlight_data.index:
            highlight_set = highlight_data[row_idx]
            for col_name in highlight_set:
                if col_name in row.index:
                    styles[row.index.get_loc(col_name)] = 'background-color: #ffcccc'
        
        return styles
    
    # 포맷 설정
    quarter_labels = generate_quarter_labels(NUM_QUARTERS)
    quarter_labels_reversed = list(reversed(quarter_labels))
    
    format_dict = {
        'Growth_Rate': lambda x: f'{x:.1f}%',  # 소수점 한자리
    }
    for q in quarter_labels_reversed:
        if q in df.columns:
            format_dict[q] = '{:,.0f}'
    
    styled = df.style.apply(style_growth_cells, axis=1)
    styled = styled.format(format_dict, na_rep='-')
    
    # 폰트 설정
    styled = styled.set_properties(**{
        'font-family': 'Pretendard, sans-serif',
        'font-size': '10pt'
    })
    
    return styled

# ==========================================
# 5. 메인 실행 블록
# ==========================================
if __name__ == "__main__":
    try:
        df_input = pd.read_csv(INPUT_FILE)
    except:
        print(f"오류: '{INPUT_FILE}' 파일을 찾을 수 없습니다.")
        exit()

    ticker_col = next((col for col in df_input.columns if col.lower() == 'ticker'), None)
    if not ticker_col: 
        print("CSV 파일에 'Ticker' 헤더가 없습니다.")
        exit()
    
    # Company 컬럼 찾기 (선택사항)
    company_col = None
    for col in df_input.columns:
        if col.lower() in ['company', 'company name', 'name']:
            company_col = col
            break
    
    # 티커와 기업명을 튜플로 생성
    if company_col:
        ticker_list = list(zip(df_input[ticker_col].tolist(), df_input[company_col].tolist()))
        print(f"Company 컬럼 발견: '{company_col}' - 기업명 기반 검색 활성화")
    else:
        ticker_list = df_input[ticker_col].tolist()
        print("Company 컬럼 없음 - 티커만으로 검색")
        
    print(f"총 {len(ticker_list)}개 기업 분석 시작 (병렬 처리 / 전 세계 거래소 지원 / {NUM_QUARTERS}분기)...")
    print("매출 + 영업이익 데이터를 동시에 수집합니다.\n")

    success_data = []
    failed_data = []
    
    # 병렬 처리 실행
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(executor.map(process_ticker, ticker_list))
    
    # 결과 분류
    for res in results:
        if res['status'] == 'success':
            del res['status']
            success_data.append(res)
        else:
            # 티커 정보 추출 (튜플인 경우 첫 번째 요소)
            failed_ticker = res['ticker'][0] if isinstance(res['ticker'], tuple) else res['ticker']
            failed_data.append({'Ticker': failed_ticker, 'Reason': res['reason']})
            print(f"[{failed_ticker}] 제외됨 ({res['reason']})")

    # yfinance로 산업 정보 조회
    if success_data:
        print("\n산업 정보 조회 중...")
        for data in success_data:
            ticker = data['ticker']
            industry = get_industry(ticker)
            data['industry'] = industry
            print(f"  [{ticker}] {industry}")
            time.sleep(0.2)

    # ---------------------------------------------------------
    # 데이터프레임 생성 및 저장
    # ---------------------------------------------------------
    if success_data:
        # 매출 데이터프레임
        df_revenue = create_dataframe(success_data, 'revenue')
        df_revenue = df_revenue.sort_values('Is_Before_4Q25', ascending=True)
        
        # 영업이익 데이터프레임
        df_operating = create_dataframe(success_data, 'operating')
        df_operating = df_operating.sort_values('Is_Before_4Q25', ascending=True)
        
        # 최종 컬럼 정렬 (Is_Before_4Q25, Highlight_Quarters 제외)
        quarter_labels = generate_quarter_labels(NUM_QUARTERS)
        quarter_labels_reversed = list(reversed(quarter_labels))
        final_cols = ['Ticker', 'Industry', 'Latest_Date', 'Growth_Rate'] + quarter_labels_reversed
        
        # 존재하는 컬럼만 선택
        revenue_cols = [c for c in final_cols if c in df_revenue.columns]
        operating_cols = [c for c in final_cols if c in df_operating.columns]
        
        df_revenue_final = df_revenue[revenue_cols].copy()
        df_operating_final = df_operating[operating_cols].copy()
        
        # 스타일 적용
        styled_revenue = apply_styling(df_revenue_final)
        styled_operating = apply_styling(df_operating_final)
        
        # 엑셀 파일에 두 시트로 저장 (B2부터 시작)
        with pd.ExcelWriter(OUTPUT_SUCCESS, engine='openpyxl') as writer:
            styled_revenue.to_excel(writer, sheet_name='Revenue', index=False, startrow=1, startcol=1)
            styled_operating.to_excel(writer, sheet_name='Operating Income', index=False, startrow=1, startcol=1)
        
        print(f"\n✅ [성공] 통합 파일 저장: '{OUTPUT_SUCCESS}'")
        print(f"   - Revenue 시트: {len(df_revenue_final)}개 종목")
        print(f"   - Operating Income 시트: {len(df_operating_final)}개 종목")
        print(f"   - 4Q25 이전 데이터는 각 시트 하단에 위치합니다.")
        print(f"   - 각 분기별 롤링 성장률 10% 이상인 셀은 연한 빨간색으로 표시됩니다.")

    # ---------------------------------------------------------
    # 실패 파일 저장 (수동 작업용)
    # ---------------------------------------------------------
    if failed_data:
        df_failed = pd.DataFrame(failed_data)
        df_failed.to_excel(OUTPUT_FAILED, index=False)
        print(f"\n⚠️ [실패] 오류 티커 리스트: '{OUTPUT_FAILED}' 저장 완료")
        print(f"   - 총 {len(failed_data)}개 티커가 제외되었습니다.")
        print(" -> 이 파일에 있는 기업들만 수동으로 확인하세요.")

Company 컬럼 발견: 'Company' - 기업명 기반 검색 활성화
총 156개 기업 분석 시작 (병렬 처리 / 전 세계 거래소 지원 / 17분기)...
매출 + 영업이익 데이터를 동시에 수집합니다.

[ASML] 성공 (NMS → US) - 매출: 1.4%, 영업이익: 0.7%)
[MSFT] 성공 (NMS → US) - 매출: 4.0%, 영업이익: 4.9%)


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: A000660"}}}


[META] 성공 (NMS → US) - 매출: 6.1%, 영업이익: 3.5%)
[TSLA] 성공 (NMS → US) - 매출: -0.8%, 영업이익: -0.2%)


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: A000660.NS"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: A000660.BO"}}}


[LRCX] 성공 (NMS → US) - 매출: 4.9%, 영업이익: 7.4%)
[APH] 성공 (NYQ → US) - 매출: 10.1%, 영업이익: 15.3%)
[IBM] 성공 (NYQ → US) - 매출: 3.3%, 영업이익: 6.2%)
[T] 성공 (NYQ → US) - 매출: 0.9%, 영업이익: 0.9%)
[NOW] 성공 (NYQ → US) - 매출: 4.8%, 영업이익: 2.0%)
[DHR] 성공 (NYQ → US) - 매출: -26.9%, 영업이익: -28.2%)


...

  [5440] N/A
  [6104] N/A
  [ETD] Furnishings, Fixtures & Appliances
  [ALRS] Banks - Regional
  [MNRO] Auto Parts


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: 6345"}}}


  [6345] N/A
  [WASH] Banks - Regional
  [HONE] N/A


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: 4552"}}}


  [4552] N/A
  [CARERATING] N/A

✅ [성공] 통합 파일 저장: '260128_해외빨간줄.xlsx'
   - Revenue 시트: 112개 종목
   - Operating Income 시트: 112개 종목
   - 4Q25 이전 데이터는 각 시트 하단에 위치합니다.
   - 각 분기별 롤링 성장률 10% 이상인 셀은 연한 빨간색으로 표시됩니다.

⚠️ [실패] 오류 티커 리스트: 'failed_tickers.xlsx' 저장 완료
   - 총 44개 티커가 제외되었습니다.
 -> 이 파일에 있는 기업들만 수동으로 확인하세요.


In [1]:
import pandas as pd
import requests
import time
import random
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime
import concurrent.futures
import yfinance as yf

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
INPUT_FILE = "data.csv"               # 입력 파일명
OUTPUT_SUCCESS = "Today.xlsx"  # 성공한 미국 주식 저장 (매출)
OUTPUT_FAILED = "non_us.xlsx"   # 실패/비미국 주식 저장 (수동 확인용)

FRESHNESS_THRESHOLD = 110  # 110일(약 3.5개월) 지났으면 '미반영' 처리
MAX_WORKERS = 4            # 병렬 처리 개수 (너무 높이면 차단됨)
NUM_QUARTERS = 17          # 가져올 분기 수

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}

# 숫자 변환 헬퍼 함수
def parse_money_string(value_str):
    if not isinstance(value_str, str): return value_str
    s = value_str.strip().replace(',', '')
    if s == '-': return 0
    try:
        if s.endswith('B'): return float(s[:-1]) * 1_000_000_000
        elif s.endswith('M'): return float(s[:-1]) * 1_000_000
        elif s.endswith('K'): return float(s[:-1]) * 1_000
        elif s.endswith('%'): return float(s[:-1])
        else: return float(s)
    except: return 0

# 분기 라벨 생성 (최신 분기 = 4Q25 기준)
def generate_quarter_labels(num_quarters):
    """
    최신 분기를 4Q25로 시작해서 역순으로 라벨 생성
    예: ['4Q25', '3Q25', '2Q25', '1Q25', '4Q24', '3Q24', ...]
    """
    labels = []
    year = 25
    quarter = 4
    
    for _ in range(num_quarters):
        labels.append(f"{quarter}Q{year}")
        quarter -= 1
        if quarter == 0:
            quarter = 4
            year -= 1
    
    return labels

# yfinance에서 산업 정보 가져오기
def get_industry(ticker):
    """yfinance를 사용해 티커의 산업(industry) 정보 가져오기"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        # 'industry' 또는 'sector' 정보 반환
        industry = info.get('industry', info.get('sector', 'N/A'))
        return industry
    except:
        return 'N/A'

# ==========================================
# 2. 개별 기업 처리 함수 (Worker)
# ==========================================
def process_ticker(raw_ticker):
    # 티커 전처리
    ticker = str(raw_ticker).strip().replace('.', '-').replace(' ', '-').lower()
    
    # 한국 주식/숫자 티커 필터링
    if any(char.isdigit() for char in ticker) and not ticker.isalpha(): 
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Korea/Number Stock'}

    # [핵심] 분기 데이터 강제 경로 (?p=quarterly)
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"
    
    retry_count = 0
    time.sleep(random.uniform(1.0, 3.0)) # 기본 대기

    # 접속 시도
    while retry_count < 3:
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code == 200:
                break
            elif response.status_code == 404:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Not US Stock (404)'}
            elif response.status_code == 429: # 과부하 시 대기
                time.sleep(random.uniform(10, 20))
                retry_count += 1
            else:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Error {response.status_code}'}
        except:
            retry_count += 1
            time.sleep(2)
            
    if retry_count >= 3:
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Connection Timeout'}

    # 데이터 파싱
    try:
        dfs = pd.read_html(StringIO(response.text))
        if not dfs: return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'No Table Found'}
        df_fin = dfs[0]

        # 날짜 컬럼 확인
        date_cols = df_fin.columns[1:].tolist()
        latest_date_str = date_cols[0]
        
        # 날짜 및 연간 데이터 검증
        freshness_diff = 0
        is_before_4q25 = False
        try:
            d1 = pd.to_datetime(latest_date_str)
            d2 = pd.to_datetime(date_cols[1])
            
            # 1. 연간 데이터(FY) 체크: 간격이 250일 이상이면 실패 처리
            if abs((d1 - d2).days) > 250:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Received Annual Data (FY)'}
            
            # 2. 최신성 체크
            freshness_diff = (datetime.now() - d1).days
            
            # 3. 4Q25 이전 체크 - Latest_Date가 "Q4 2025" 형식인지 확인
            # 날짜 문자열에서 분기 정보 추출
            date_str_lower = latest_date_str.lower()
            if 'q4' in date_str_lower and '2025' in date_str_lower:
                is_before_4q25 = False
            elif '2025' in date_str_lower:
                is_before_4q25 = True  # Q1, Q2, Q3 2025
            elif '2024' in date_str_lower or int(latest_date_str.split('-')[0]) < 2025:
                is_before_4q25 = True
            else:
                is_before_4q25 = False
        except:
            pass # 날짜 파싱 에러나도 일단 진행

        # 매출 데이터 찾기
        target_rows = ["Revenue"]
        revenue_row = pd.DataFrame()
        for metric in target_rows:
            temp = df_fin[df_fin.iloc[:, 0].str.contains(metric, case=False, na=False)]
            if not temp.empty: revenue_row = temp; break
        
        if revenue_row.empty: return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Metric Not Found'}
        
        revenue_values = [parse_money_string(v) for v in revenue_row.iloc[0, 1:].tolist()]
        if len(revenue_values) < NUM_QUARTERS: 
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Data Shortage (<{NUM_QUARTERS}Q)'}

        # **순서 반전: 최근 분기를 오른쪽에 배치**
        revenue_values_reversed = list(reversed(revenue_values[:NUM_QUARTERS]))

        # 성장률 계산 (최신 4분기 vs 직전 4분기)
        recent_avg = sum(revenue_values[0:4]) / 4
        past_avg   = sum(revenue_values[1:5]) / 4
        growth = (recent_avg / past_avg) - 1 if past_avg != 0 else 0

        # 성공 데이터 반환
        row = {
            'status': 'success',
            'Ticker': raw_ticker.upper(),
            'Industry': 'PENDING',  # 나중에 일괄 조회
            'Latest_Date': latest_date_str,
            'Growth_Rate': growth * 100,  # 백분율로 변환
            'Recent_Avg': recent_avg,
            'Is_Before_4Q25': is_before_4q25,  # 정렬용
        }
        
        # 분기 라벨 생성 (4Q25부터 시작)
        quarter_labels = generate_quarter_labels(NUM_QUARTERS)
        # 역순 배치: 4Q21(왼쪽) -> 4Q25(오른쪽)
        quarter_labels_reversed = list(reversed(quarter_labels))
        
        # 분기 데이터 추가
        for i in range(NUM_QUARTERS):
            if i < len(revenue_values_reversed): 
                row[quarter_labels_reversed[i]] = revenue_values_reversed[i]
            
        print(f"[{raw_ticker.upper()}] 성공 (매출 성장률 {growth*100:.1f}%)")
        return row

    except Exception as e:
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': str(e)}

# ==========================================
# 3. 메인 실행 블록
# ==========================================
if __name__ == "__main__":
    try:
        df_input = pd.read_csv(INPUT_FILE)
    except:
        print(f"오류: '{INPUT_FILE}' 파일을 찾을 수 없습니다.")
        exit()

    ticker_col = next((col for col in df_input.columns if col.lower() == 'ticker'), None)
    if not ticker_col: 
        print("CSV 파일에 'Ticker' 헤더가 없습니다.")
        exit()
        
    ticker_list = df_input[ticker_col].tolist()
    
    print(f"총 {len(ticker_list)}개 기업 매출 분석 시작 (병렬 처리 / 분기 데이터 강제 / {NUM_QUARTERS}분기)...")

    success_data = []
    failed_data = []
    
    # 병렬 처리 실행
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(executor.map(process_ticker, ticker_list))
    
    # 결과 분류
    for res in results:
        if res['status'] == 'success':
            del res['status']
            success_data.append(res)
        else:
            failed_data.append({'Ticker': res['ticker'], 'Reason': res['reason']})
            print(f"[{res['ticker']}] 제외됨 ({res['reason']})")

    # ---------------------------------------------------------
    # 4-1. 성공 파일 저장 (미국 주식)
    # ---------------------------------------------------------
    if success_data:
        df_success = pd.DataFrame(success_data)
        
        # yfinance로 산업 정보 조회
        print("\n산업 정보 조회 중...")
        for idx, row in df_success.iterrows():
            ticker = row['Ticker']
            industry = get_industry(ticker)
            df_success.at[idx, 'Industry'] = industry
            print(f"  [{ticker}] {industry}")
            time.sleep(0.2)  # API 호출 간격
        
        # 4Q25 이전 데이터를 아래로 정렬
        df_success = df_success.sort_values('Is_Before_4Q25', ascending=True)
        
        # 최종 출력 컬럼 선택
        final_cols = ['Ticker', 'Industry', 'Latest_Date', 'Growth_Rate', 'Recent_Avg']
        
        # 분기 컬럼들 추가 (4Q21 -> 4Q25 순서)
        quarter_labels = generate_quarter_labels(NUM_QUARTERS)
        quarter_labels_reversed = list(reversed(quarter_labels))
        
        for q_label in quarter_labels_reversed:
            if q_label in df_success.columns:
                final_cols.append(q_label)
        
        # 정렬 컬럼 제거
        df_final = df_success[final_cols].copy()
        
        # 롤링 성장률 체크 함수 - 슬라이딩 윈도우 방식
        def check_rolling_growth(row):
            """각 분기마다 슬라이딩 윈도우로 성장률 체크 (최근4분기 vs 이전4분기, 1개씩 겹침)"""
            highlight_quarters = set()  # 음영 처리할 분기들
            
            try:
                # 분기 컬럼만 추출 (왼쪽=과거, 오른쪽=최신)
                q_cols = [col for col in row.index if 'Q' in col and col[0].isdigit()]
                n = len(q_cols)
                
                if n >= 5:
                    # 각 위치에서 체크 (최소 5개 분기 필요)
                    # 오른쪽부터 왼쪽으로 이동하면서 체크
                    for position in range(0, n - 4):
                        # 해당 분기의 실제 인덱스 (가장 최근 분기)
                        current_idx = n - 1 - position
                        
                        # 최근 4분기: current_idx부터 왼쪽으로 3개 더 (총 4개)
                        # 예: 4Q25, 3Q25, 2Q25, 1Q25
                        recent_4q = [row[q_cols[current_idx - i]] for i in range(4)]
                        
                        # 이전 4분기: current_idx-1부터 왼쪽으로 3개 더 (총 4개, 1개씩 겹침)
                        # 예: 3Q25, 2Q25, 1Q25, 4Q24
                        prev_4q = [row[q_cols[current_idx - 1 - i]] for i in range(4)]
                        
                        recent_avg = sum(recent_4q) / 4
                        prev_avg = sum(prev_4q) / 4
                        
                        if prev_avg != 0:
                            growth_ratio = recent_avg / prev_avg
                            if growth_ratio >= 1.1:
                                # 해당 분기만 하이라이트
                                highlight_quarters.add(q_cols[current_idx])
            except Exception as e:
                pass
            
            return highlight_quarters
        
        # 각 행에 대해 하이라이트할 분기 계산
        df_final['Highlight_Quarters'] = df_final.apply(check_rolling_growth, axis=1)
        
        # 스타일 함수
        def style_growth_cells(row):
            styles = ['' for _ in row.index]
            
            if 'Highlight_Quarters' in row.index and row['Highlight_Quarters']:
                highlight_set = row['Highlight_Quarters']
                for col_name in highlight_set:
                    if col_name in row.index:
                        styles[row.index.get_loc(col_name)] = 'background-color: #ffcccc'
            
            return styles
        
        # Highlight_Quarters 컬럼 제거 (표시용이므로)
        display_cols = [col for col in df_final.columns if col != 'Highlight_Quarters']
        df_display = df_final[display_cols].copy()
        
        # 포맷 설정
        format_dict = {
            'Recent_Avg': '{:,.0f}',
            'Growth_Rate': '{:.2f}%',  # 백분율 표시
        }
        for q in quarter_labels_reversed:
            if q in df_display.columns:
                format_dict[q] = '{:,.0f}'
        
        # 스타일 적용 (원본 df_final 사용 - Highlight_Quarters 포함)
        styled = df_final.style.apply(style_growth_cells, axis=1)
        styled = styled.format(format_dict, precision=0)
        
        # Highlight_Quarters 컬럼 숨기기
        try:
            if hasattr(styled, "hide"):
                styled = styled.hide(['Highlight_Quarters'], axis="columns")
            elif hasattr(styled, "hide_columns"):
                styled = styled.hide_columns(['Highlight_Quarters'])
        except:
            pass

        # 엑셀 저장
        styled.to_excel(OUTPUT_SUCCESS, index=False, engine='openpyxl')
        print(f"\n✅ [성공] 미국 주식 매출 데이터: '{OUTPUT_SUCCESS}' 저장 완료")
        print(f"   - 4Q25 이전 데이터는 파일 하단에 위치합니다.")
        print(f"   - 각 분기별 롤링 성장률 10% 이상인 셀은 연한 빨간색으로 표시됩니다.")

    # ---------------------------------------------------------
    # 4-2. 실패 파일 저장 (수동 작업용)
    # ---------------------------------------------------------
    if failed_data:
        df_failed = pd.DataFrame(failed_data)
        df_failed.to_excel(OUTPUT_FAILED, index=False)
        print(f"⚠️ [제외] 비미국/오류 리스트: '{OUTPUT_FAILED}' 저장 완료")
        print(" -> 이 파일에 있는 기업들만 수동으로 확인하세요.")

오류: '260126_Earnings.csv' 파일을 찾을 수 없습니다.


NameError: name 'df_input' is not defined

In [1]:
import pandas as pd
import requests
import time
import random
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime
import concurrent.futures
import yfinance as yf

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
INPUT_FILE = "data.csv"               # 입력 파일명
OUTPUT_SUCCESS = "Today.xlsx"  # 통합 파일 (매출 + 영업이익)
OUTPUT_FAILED = "failed_tickers.xlsx"   # 실패 티커 저장 (수동 확인용)

FRESHNESS_THRESHOLD = 110  # 110일(약 3.5개월) 지났으면 '미반영' 처리
MAX_WORKERS = 4            # 병렬 처리 개수 (너무 높이면 차단됨)
NUM_QUARTERS = 17          # 가져올 분기 수

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}

# 숫자 변환 헬퍼 함수
def parse_money_string(value_str):
    if not isinstance(value_str, str): return value_str
    s = value_str.strip().replace(',', '')
    if s == '-': return 0
    try:
        if s.endswith('B'): return float(s[:-1]) * 1_000_000_000
        elif s.endswith('M'): return float(s[:-1]) * 1_000_000
        elif s.endswith('K'): return float(s[:-1]) * 1_000
        elif s.endswith('%'): return float(s[:-1])
        else: return float(s)
    except: return 0

# 분기 라벨 생성 (최신 분기 = 4Q25 기준)
def generate_quarter_labels(num_quarters):
    """
    최신 분기를 4Q25로 시작해서 역순으로 라벨 생성
    예: ['4Q25', '3Q25', '2Q25', '1Q25', '4Q24', '3Q24', ...]
    """
    labels = []
    year = 25
    quarter = 4
    
    for _ in range(num_quarters):
        labels.append(f"{quarter}Q{year}")
        quarter -= 1
        if quarter == 0:
            quarter = 4
            year -= 1
    
    return labels

# 거래소 코드 → 국가 매핑
def get_country_from_exchange(sa_exchange):
    """Stock Analysis 거래소 코드로 국가 반환"""
    exchange_to_country = {
        None: 'United States',  # 미국 (기본)
        'sto': 'Sweden',
        'epa': 'France',
        'ams': 'Netherlands',
        'bru': 'Belgium',
        'lse': 'United Kingdom',
        'fra': 'Germany',
        'swx': 'Switzerland',
        'tyo': 'Japan',
        'hkg': 'Hong Kong',
        'tai': 'Taiwan',
        'shh': 'China',
        'shz': 'China',
        'idx': 'Indonesia',
        'nse': 'India',
        'bse': 'India',
        'bmv': 'Mexico',
        'sao': 'Brazil',
        'jse': 'South Africa',
        'tase': 'Israel',
    }
    return exchange_to_country.get(sa_exchange, 'Unknown')

# yfinance에서 산업 정보만 가져오기
def get_industry_info(ticker):
    """yfinance를 사용해 티커의 산업 정보 가져오기"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Industry
        industry = info.get('industry') or info.get('sector', 'N/A')
        
        return industry
    except:
        return 'N/A'

# yfinance로 거래소 확인 및 Stock Analysis 거래소 코드 반환
def get_stock_analysis_exchange(ticker, company_name=None):
    """
    yfinance로 거래소 확인 후 Stock Analysis URL 형식 반환
    Returns: (sa_exchange_code, original_exchange, matched_ticker)
    """
    # yfinance 거래소 → Stock Analysis 거래소 코드 매핑
    exchange_map = {
        # 미국 거래소 (기본 URL)
        'NMS': None, 'NYQ': None, 'ASE': None, 'PCX': None, 'NGM': None, 'NAS': None,
        
        # 유럽
        'STO': 'sto', 'PAR': 'epa', 'AMS': 'ams', 'BRU': 'bru', 'LSE': 'lse', 'FRA': 'fra', 'SWX': 'swx',
        
        # 아시아 (한국 제외)
        'JPX': 'tyo', 'TYO': 'tyo', 'HKG': 'hkg',
        'TAI': 'tai', 'SHH': 'shh', 'SHZ': 'shz', 'IDX': 'idx', 'NSI': 'nse', 'BOM': 'bse',
        
        # 한국 (스킵 표시)
        'KSC': 'SKIP_KOREA', 'KOE': 'SKIP_KOREA',
        
        # 기타
        'BMV': 'bmv', 'SAO': 'sao', 'JNB': 'jse', 'TAD': 'tase',
    }
    
    # 스마트 우선순위: 티커 패턴 기반
    search_attempts = [ticker]  # 원본
    
    # 숫자만 있는 티커 (아시아 우선)
    if ticker.replace(' ', '').replace('-', '').isdigit():
        ticker_clean = ticker.replace(' ', '')
        if len(ticker_clean) <= 4:
            # 4자리 이하 → 도쿄 우선
            search_attempts.extend([f"{ticker_clean}.T", f"{ticker_clean}.HK"])
        else:
            # 5자리 이상 → 홍콩, 인도, 중동
            search_attempts.extend([f"{ticker_clean}.HK", f"{ticker_clean}.NS", f"{ticker_clean}.BO", f"{ticker_clean}.SR"])
    
    # 공백 포함 티커 (스웨덴 클래스주)
    elif ' ' in ticker:
        ticker_hyphen = ticker.replace(' ', '-')
        search_attempts.extend([f"{ticker_hyphen}.ST", f"{ticker}.ST"])
    
    # 한국 주식은 검색하지 않음
    # 대문자 단어 티커 (인도 우선 - 티커명이 곧 기업명)
    elif ticker.isupper() and len(ticker) > 4:
        search_attempts.extend([f"{ticker}.NS", f"{ticker}.BO"])
    
    # 일반 티커 - 주요 거래소만 (한국 제외)
    else:
        search_attempts.extend([
            f"{ticker}.ST",  # Stockholm
            f"{ticker}.PA",  # Paris
            f"{ticker}.L",   # London
            f"{ticker}.T",   # Tokyo
            f"{ticker}.HK",  # Hong Kong
        ])
    
    # 각 티커 시도
    for attempt_ticker in search_attempts:
        try:
            stock = yf.Ticker(attempt_ticker)
            info = stock.info
            
            # 유효한 데이터 확인
            symbol = info.get('symbol')
            exchange = info.get('exchange', '')
            
            # 데이터가 있고 거래소 정보가 있으면 성공
            if symbol and exchange:
                sa_code = exchange_map.get(exchange)
                
                # 한국 거래소 체크
                if sa_code == 'SKIP_KOREA':
                    return 'SKIP_KOREA', exchange, attempt_ticker
                
                return sa_code, exchange, attempt_ticker
                
        except Exception as e:
            continue
    
    # 모든 시도 실패
    return None, None, ticker

# ==========================================
# 2. 개별 기업 처리 함수 (Worker)
# ==========================================
def process_ticker(ticker_data):
    # ticker_data는 (ticker, company_name) 튜플
    if isinstance(ticker_data, tuple):
        raw_ticker, company_name = ticker_data
    else:
        raw_ticker = ticker_data
        company_name = None
    
    # 티커 전처리
    ticker = str(raw_ticker).strip().replace('.', '-').replace(' ', '-').lower()
    
    # yfinance로 거래소 확인 (기업명 포함)
    sa_exchange, original_exchange, matched_ticker = get_stock_analysis_exchange(raw_ticker, company_name)
    
    # 한국 거래소는 제외 (별도 툴 사용)
    if sa_exchange in ['ksc', 'koe']:
        return 'SKIP_KOREA', original_exchange, matched_ticker
    
    # 거래소 정보 로깅
    exchange_info = f"{original_exchange} → {sa_exchange if sa_exchange else 'US'}"
    if matched_ticker != raw_ticker:
        exchange_info += f" (matched: {matched_ticker})"
    
    # 한국 주식 필터링 확인
    if exchange_info == 'SKIP_KOREA':
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Korean Stock (Skipped - separate tool)'}
    
    # 숫자만 있는 티커 중 한국 가능성 체크 (A로 시작하는 6자리)
    if isinstance(raw_ticker, str) and raw_ticker.startswith('A') and len(raw_ticker) == 7 and raw_ticker[1:].isdigit():
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Korean Stock Code (Skipped)'}
    
    # 한국 주식/숫자만 있는 티커 필터링 (단, 일본 주식은 허용)
    if sa_exchange not in ['tyo', None]:  # 일본이나 미국이 아니면
        if any(char.isdigit() for char in ticker) and not ticker.isalpha():
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Non-supported number ticker ({exchange_info})'}

    # Stock Analysis URL 생성 (거래소별)
    if sa_exchange:
        # 해외 거래소 - matched_ticker 사용 (접미사 포함 가능)
        # Stock Analysis는 접미사 없이 사용하므로 원본 티커 사용
        url = f"https://stockanalysis.com/quote/{sa_exchange}/{raw_ticker.upper().replace(' ', '-')}/financials/?p=quarterly"
    else:
        # 미국 거래소 (기본)
        url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"
    
    retry_count = 0
    time.sleep(random.uniform(1.0, 3.0)) # 기본 대기

    # 접속 시도
    while retry_count < 3:
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code == 200:
                break
            elif response.status_code == 404:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'404 Not Found ({exchange_info})'}
            elif response.status_code == 429: # 과부하 시 대기
                time.sleep(random.uniform(10, 20))
                retry_count += 1
            else:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Error {response.status_code}'}
        except:
            retry_count += 1
            time.sleep(2)
            
    if retry_count >= 3:
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Connection Timeout'}

    # 데이터 파싱
    try:
        dfs = pd.read_html(StringIO(response.text))
        if not dfs: return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'No Table Found'}
        df_fin = dfs[0]

        # 날짜 컬럼 확인
        date_cols = df_fin.columns[1:].tolist()
        
        # Stock Analysis는 헤더가 2행일 수 있음
        # 1행: Q3 2025, Q4 2025...
        # 2행: Sep '25 Sep 30, 2025...
        # 실제 날짜가 있는 행 찾기
        latest_date_str = date_cols[0]
        if isinstance(latest_date_str, tuple):
            # 튜플인 경우 두 번째 요소가 실제 날짜
            latest_date_str = latest_date_str[1] if len(latest_date_str) > 1 else str(latest_date_str[0])
        
        # 날짜 컬럼들도 동일하게 처리
        processed_date_cols = []
        for col in date_cols:
            if isinstance(col, tuple):
                processed_date_cols.append(col[1] if len(col) > 1 else str(col[0]))
            else:
                processed_date_cols.append(col)
        
        # 날짜 및 연간 데이터 검증
        freshness_diff = 0
        is_before_4q25 = False
        try:
            d1 = pd.to_datetime(latest_date_str, format='mixed')
            
            # 두 번째 날짜도 처리
            if len(processed_date_cols) > 1:
                d2 = pd.to_datetime(processed_date_cols[1], format='mixed')
            else:
                d2 = d1
            
            # 1. 연간 데이터(FY) 체크: 간격이 250일 이상이면 실패 처리
            if abs((d1 - d2).days) > 250:
                return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'Received Annual Data (FY)'}
            
            # 2. 최신성 체크
            freshness_diff = (datetime.now() - d1).days
            
            # 3. 4Q25 이전 체크 - Latest_Date가 2025년 10월 1일 이전인지 확인
            cutoff_date = datetime(2025, 10, 1)
            is_before_4q25 = (d1 < cutoff_date)
        except:
            pass # 날짜 파싱 에러나도 일단 진행

        # ========================================
        # 매출 데이터 찾기 (정확히 일치하는 경우만)
        # ========================================
        revenue_target = ["Revenue", "Total Revenue", "Net Revenue", "Sales"]
        revenue_row = pd.DataFrame()
        for metric in revenue_target:
            # 정확히 일치하는 경우만 찾기 (strip으로 공백 제거 후 비교)
            temp = df_fin[df_fin.iloc[:, 0].str.strip().str.lower() == metric.lower()]
            if not temp.empty: 
                revenue_row = temp
                break
        
        if revenue_row.empty:
            revenue_values = None
            revenue_growth = 0
            revenue_avg = 0
        else:
            revenue_values = [parse_money_string(v) for v in revenue_row.iloc[0, 1:].tolist()]
            if len(revenue_values) >= 5:
                recent_avg = sum(revenue_values[0:4]) / 4
                past_avg = sum(revenue_values[1:5]) / 4
                revenue_growth = (recent_avg / past_avg) - 1 if past_avg != 0 else 0
                revenue_avg = recent_avg
            else:
                revenue_growth = 0
                revenue_avg = 0

        # ========================================
        # 영업이익 데이터 찾기
        # ========================================
        op_target = ["Operating Income", "Operating Profit", "Pretax Income", "Net Income"]
        op_row = pd.DataFrame()
        for metric in op_target:
            temp = df_fin[df_fin.iloc[:, 0].str.contains(metric, case=False, na=False)]
            if not temp.empty: 
                op_row = temp
                break
        
        if op_row.empty:
            op_values = None
            op_growth = 0
            op_avg = 0
        else:
            op_values = [parse_money_string(v) for v in op_row.iloc[0, 1:].tolist()]
            if len(op_values) >= 5:
                recent_avg = sum(op_values[0:4]) / 4
                past_avg = sum(op_values[1:5]) / 4
                op_growth = (recent_avg / past_avg) - 1 if past_avg != 0 else 0
                op_avg = recent_avg
            else:
                op_growth = 0
                op_avg = 0

        # 둘 다 없으면 실패
        if revenue_values is None and op_values is None:
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': 'No Revenue or OpIncome Data'}
        
        # 분기 수 체크
        if revenue_values and len(revenue_values) < NUM_QUARTERS:
            revenue_values = None
        if op_values and len(op_values) < NUM_QUARTERS:
            op_values = None
            
        # 둘 다 분기 부족이면 실패
        if revenue_values is None and op_values is None:
            return {'status': 'failed', 'ticker': raw_ticker, 'reason': f'Data Shortage (<{NUM_QUARTERS}Q)'}

        # 성공 데이터 반환
        result = {
            'status': 'success',
            'ticker': raw_ticker.upper(),
            'matched_ticker': matched_ticker,  # Industry 조회용
            'country': get_country_from_exchange(sa_exchange),  # 거래소 기반 국가
            'industry': 'PENDING',
            'latest_date': latest_date_str,
            'is_before_4q25': is_before_4q25,
            'revenue_values': revenue_values[:NUM_QUARTERS] if revenue_values else None,
            'revenue_growth': revenue_growth,
            'revenue_avg': revenue_avg,
            'op_values': op_values[:NUM_QUARTERS] if op_values else None,
            'op_growth': op_growth,
            'op_avg': op_avg,
        }
        
        print(f"[{raw_ticker.upper()}] 성공 ({exchange_info}) - 매출: {revenue_growth*100:.1f}%, 영업이익: {op_growth*100:.1f}%)")
        return result

    except Exception as e:
        return {'status': 'failed', 'ticker': raw_ticker, 'reason': str(e)}

# ==========================================
# 3. 데이터프레임 생성 함수
# ==========================================
def create_dataframe(success_data, data_type='revenue'):
    """
    data_type: 'revenue' 또는 'operating'
    """
    rows = []
    quarter_labels = generate_quarter_labels(NUM_QUARTERS)
    quarter_labels_reversed = list(reversed(quarter_labels))
    
    for data in success_data:
        if data_type == 'revenue':
            if data['revenue_values'] is None:
                continue
            values = data['revenue_values']
            growth = data['revenue_growth']
            avg = data['revenue_avg']
        else:  # operating
            if data['op_values'] is None:
                continue
            values = data['op_values']
            growth = data['op_growth']
            avg = data['op_avg']
        
        # 순서 반전: 최근 분기를 오른쪽에
        values_reversed = list(reversed(values))
        
        row = {
            'Ticker': data['ticker'],
            'Country': data['country'],
            'Industry': data['industry'],
            'Latest_Date': data['latest_date'],
            'Growth_Rate': growth * 100,
            'Is_Before_4Q25': data['is_before_4q25'],
        }
        
        # 분기 데이터 추가
        for i, q_label in enumerate(quarter_labels_reversed):
            if i < len(values_reversed):
                row[q_label] = values_reversed[i]
        
        rows.append(row)
    
    return pd.DataFrame(rows)

# ==========================================
# 4. 스타일 적용 함수
# ==========================================
def apply_styling(df_input):
    """롤링 성장률 체크 및 스타일 적용"""
    df = df_input.copy()
    
    # 롤링 성장률 체크 함수
    def check_rolling_growth(row):
        highlight_quarters = set()
        
        try:
            q_cols = [col for col in row.index if 'Q' in col and col[0].isdigit()]
            n = len(q_cols)
            
            if n >= 5:
                for position in range(0, n - 4):
                    current_idx = n - 1 - position
                    recent_4q = [row[q_cols[current_idx - i]] for i in range(4)]
                    prev_4q = [row[q_cols[current_idx - 1 - i]] for i in range(4)]
                    
                    recent_avg = sum(recent_4q) / 4
                    prev_avg = sum(prev_4q) / 4
                    
                    if prev_avg != 0:
                        growth_ratio = recent_avg / prev_avg
                        if growth_ratio >= 1.1:
                            highlight_quarters.add(q_cols[current_idx])
        except:
            pass
        
        return highlight_quarters
    
    # 각 행에 하이라이트할 분기 저장
    highlight_data = df.apply(check_rolling_growth, axis=1)
    
    # 스타일 함수
    def style_growth_cells(row):
        styles = ['' for _ in row.index]
        row_idx = row.name
        
        if row_idx in highlight_data.index:
            highlight_set = highlight_data[row_idx]
            for col_name in highlight_set:
                if col_name in row.index:
                    styles[row.index.get_loc(col_name)] = 'background-color: #ffcccc'
        
        return styles
    
    # 포맷 설정
    quarter_labels = generate_quarter_labels(NUM_QUARTERS)
    quarter_labels_reversed = list(reversed(quarter_labels))
    
    format_dict = {
        'Growth_Rate': lambda x: f'{x:.1f}%',  # 소수점 한자리
    }
    for q in quarter_labels_reversed:
        if q in df.columns:
            format_dict[q] = '{:,.0f}'
    
    styled = df.style.apply(style_growth_cells, axis=1)
    styled = styled.format(format_dict, na_rep='-')
    
    # 폰트 설정
    styled = styled.set_properties(**{
        'font-family': 'Pretendard, sans-serif',
        'font-size': '10pt'
    })
    
    return styled

# ==========================================
# 5. 메인 실행 블록
# ==========================================
if __name__ == "__main__":
    try:
        df_input = pd.read_csv(INPUT_FILE)
    except:
        print(f"오류: '{INPUT_FILE}' 파일을 찾을 수 없습니다.")
        exit()

    ticker_col = next((col for col in df_input.columns if col.lower() == 'ticker'), None)
    if not ticker_col: 
        print("CSV 파일에 'Ticker' 헤더가 없습니다.")
        exit()
    
    # Company 컬럼 찾기 (선택사항)
    company_col = None
    for col in df_input.columns:
        if col.lower() in ['company', 'company name', 'name']:
            company_col = col
            break
    
    # 티커와 기업명을 튜플로 생성
    if company_col:
        ticker_list = list(zip(df_input[ticker_col].tolist(), df_input[company_col].tolist()))
        print(f"Company 컬럼 발견: '{company_col}' - 기업명 기반 검색 활성화")
    else:
        ticker_list = df_input[ticker_col].tolist()
        print("Company 컬럼 없음 - 티커만으로 검색")
        
    print(f"총 {len(ticker_list)}개 기업 분석 시작 (병렬 처리 / 전 세계 거래소 지원 / {NUM_QUARTERS}분기)...")
    print("매출 + 영업이익 데이터를 동시에 수집합니다.\n")

    success_data = []
    failed_data = []
    
    # 병렬 처리 실행
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(executor.map(process_ticker, ticker_list))
    
    # 결과 분류
    for res in results:
        if res['status'] == 'success':
            del res['status']
            success_data.append(res)
        else:
            # 티커 정보 추출 (튜플인 경우 첫 번째 요소)
            failed_ticker = res['ticker'][0] if isinstance(res['ticker'], tuple) else res['ticker']
            failed_data.append({'Ticker': failed_ticker, 'Reason': res['reason']})
            print(f"[{failed_ticker}] 제외됨 ({res['reason']})")

    # yfinance로 산업 정보 조회
    if success_data:
        print("\n산업 정보 조회 중...")
        for data in success_data:
            ticker = data['ticker']
            industry = get_industry_info(ticker)
            data['industry'] = industry
            print(f"  [{ticker}] {industry}")
            time.sleep(0.2)

    # ---------------------------------------------------------
    # 데이터프레임 생성 및 저장
    # ---------------------------------------------------------
    if success_data:
        # 매출 데이터프레임
        df_revenue = create_dataframe(success_data, 'revenue')
        df_revenue = df_revenue.sort_values('Is_Before_4Q25', ascending=True)
        
        # 영업이익 데이터프레임
        df_operating = create_dataframe(success_data, 'operating')
        df_operating = df_operating.sort_values('Is_Before_4Q25', ascending=True)
        
        # 최종 컬럼 정렬 (Country 추가, Currency 제거)
        quarter_labels = generate_quarter_labels(NUM_QUARTERS)
        quarter_labels_reversed = list(reversed(quarter_labels))
        final_cols = ['Ticker', 'Country', 'Industry', 'Latest_Date', 'Growth_Rate'] + quarter_labels_reversed
        
        # 존재하는 컬럼만 선택
        revenue_cols = [c for c in final_cols if c in df_revenue.columns]
        operating_cols = [c for c in final_cols if c in df_operating.columns]
        
        df_revenue_final = df_revenue[revenue_cols].copy()
        df_operating_final = df_operating[operating_cols].copy()
        
        # 스타일 적용
        styled_revenue = apply_styling(df_revenue_final)
        styled_operating = apply_styling(df_operating_final)
        
        # 엑셀 파일에 두 시트로 저장 (B2부터 시작)
        with pd.ExcelWriter(OUTPUT_SUCCESS, engine='openpyxl') as writer:
            styled_revenue.to_excel(writer, sheet_name='Revenue', index=False, startrow=1, startcol=1)
            styled_operating.to_excel(writer, sheet_name='Operating Income', index=False, startrow=1, startcol=1)
        
        print(f"\n✅ [성공] 통합 파일 저장: '{OUTPUT_SUCCESS}'")
        print(f"   - Revenue 시트: {len(df_revenue_final)}개 종목")
        print(f"   - Operating Income 시트: {len(df_operating_final)}개 종목")
        print(f"   - 4Q25 이전 데이터는 각 시트 하단에 위치합니다.")
        print(f"   - 각 분기별 롤링 성장률 10% 이상인 셀은 연한 빨간색으로 표시됩니다.")

    # ---------------------------------------------------------
    # 실패 파일 저장 (수동 작업용)
    # ---------------------------------------------------------
    if failed_data:
        df_failed = pd.DataFrame(failed_data)
        df_failed.to_excel(OUTPUT_FAILED, index=False)
        print(f"\n⚠️ [실패] 오류 티커 리스트: '{OUTPUT_FAILED}' 저장 완료")
        print(f"   - 총 {len(failed_data)}개 티커가 제외되었습니다.")
        print(" -> 이 파일에 있는 기업들만 수동으로 확인하세요.")

Company 컬럼 발견: 'Company' - 기업명 기반 검색 활성화
총 325개 기업 분석 시작 (병렬 처리 / 전 세계 거래소 지원 / 17분기)...
매출 + 영업이익 데이터를 동시에 수집합니다.

[ASML] 성공 (NMS → US) - 매출: 1.4%, 영업이익: 0.7%)
[META] 성공 (NMS → US) - 매출: 6.1%, 영업이익: 3.5%)


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: A000660"}}}


[MSFT] 성공 (NMS → US) - 매출: 4.0%, 영업이익: 4.9%)


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: A000660.NS"}}}


[TSLA] 성공 (NMS → US) - 매출: -0.8%, 영업이익: -0.2%)


...

  [SHYAMCENT] N/A


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: RAJMET"}}}


  [RAJMET] N/A


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: 4679"}}}


  [4679] N/A
  [AME] Specialty Industrial Machinery
  [ABM] Specialty Business Services
  [BGS] Packaged Foods
  [T] Telecom Services

✅ [성공] 통합 파일 저장: 'Today.xlsx'
   - Revenue 시트: 150개 종목
   - Operating Income 시트: 150개 종목
   - 4Q25 이전 데이터는 각 시트 하단에 위치합니다.
   - 각 분기별 롤링 성장률 10% 이상인 셀은 연한 빨간색으로 표시됩니다.

⚠️ [실패] 오류 티커 리스트: 'failed_tickers.xlsx' 저장 완료
   - 총 175개 티커가 제외되었습니다.
 -> 이 파일에 있는 기업들만 수동으로 확인하세요.
